# Transformers

Criação de um <i>transformer</i> do zero, a partir da biblioteca `PyTorch`.

In [ ]:
%pip install torch

## Imports

In [ ]:
import math
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

## Classes
Aqui, são declaradas as classes que irão representar os blocos da figura da arquitetura do <i>transformer</i>.

![Transformer Architecture](./content/attention_architecture.png)

### <i>Multi-Head Attention</i>

Uma função de attention pode ser descrita como um mapeamento entre uma query (consulta) e um conjunto de pares key-value (chave-valor) para uma saída, onde a consulta, chaves, valores e saída são todos vetores. A saída é calculada como uma soma ponderada dos valores, onde o peso atribuído a cada valor é computado por uma função de compatibilidade entre a consulta e a chave correspondente. O bloco de multi-head attention aplica múltiplas atenções em paralelo, capturando diferentes aspectos das relações entre as palavras. Isso permite que o modelo aprenda representações mais ricas e complexas da sequência de entrada. A soma ponderada pode ser visualizada como um produto interno entre vetores, que de acordo com o artigo "Attention is All You Need", é representada por:

$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$

Os parâmetros de entrada são:
* `d_model`: dimensão da entrada;
* `num_heads`: número de <i>attention heads</i> para separar a entrada;

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()

        # ---| Verifica se o valor da dimensão da entrada é divisivel pelo número de attention heads
        assert d_model % num_heads == 0, "d_model deve ser divisível por num_heads"
        
        # ---| Dimensões
        self.d_model   = d_model              
        self.num_heads = num_heads            
        self.d_k       = d_model // num_heads # dimensão das matrizes de pesos Q, K, V
        
        # ---| Criação das matrizes de pesos Q, K, V
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)
        self.Wo = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """ 
            Este método aplica a função attention citada acima.
            Primeiro, faz-se o produto interno entre Q e K transposto, dividido pela raiz de d_k.
            Uma flag `mask` também pode ser passada em caso de precisar-se de masked attention. 
            Após o produto interno, aplica-se a função softmax para normalizar os dados e, por fim, multiplica-se esse resultado pela matriz V.
        """

        attention = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None: attention = attention.masked_fill(mask == 0, -1e9)
        attention_after_softmax = torch.softmax(attention, dim=-1)
        return torch.matmul(attention_after_softmax, V)
        
    def split_heads(self, x):
        """
            Esse método reformata a entrada para a forma (batch_size, num_heads, seq_length, d_k). 
            Isso permite ao modelo processar múltiplas heads of attention simultaneamente.
        """
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        """ 
            Após aplicar a atenção em cada head separadamente, 
            esse método combina os resultados de volta em um único tensor com a forma (batch_size, sequence_length, d_model).
        """
        batch_size, _, sequence_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, sequence_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        """ Este método cria as matrizes Q, K, V, aplica a função attention e cria a matriz de saída com a combinação da matriz attention com a matriz de saída. """
        Q = self.split_heads(self.Wq(Q))
        K = self.split_heads(self.Wk(K))
        V = self.split_heads(self.Wv(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        return self.Wo(self.combine_heads(attn_output))